In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

In [2]:
import re

In [3]:
from datetime import datetime

In [23]:
stats = [{'full': 5.0, 'trap': 2, 'distance': 24.0, 'local': 0.0, 'grade': 5.0, 'dist_trap': 14.0, 'dist_grade': 5.0, 'first_full': 0.0, 'first_trap': 3.0, 'first_dist': 6.0, 'first_local': 14.0, 'first_grade': 0.0, 'tree_full': 5.0, 'tree_trap': 13.0, 'tree_dist': 18.0, 'tree_local': 26.0, 'tree_grade': 5.0, 'mean_time': 28.182, 'min_time': 27.83, 'max_time': 28.52, 'by_mean_lost': 2.4, 'by_mean_win': 0.0, 'bends_mean': 3.3, 'position': 3.6, 'days_lr': 4.0, 'whelping': 1017.0, 'place': 1}, {'full': 1.0, 'trap': 4, 'distance': 51.0, 'local': 0.0, 'grade': 16.0, 'dist_trap': 8.0, 'dist_grade': 16.0, 'first_full': 0.0, 'first_trap': 2.0, 'first_dist': 17.0, 'first_local': 21.0, 'first_grade': 7.0, 'tree_full': 1.0, 'tree_trap': 7.0, 'tree_dist': 34.0, 'tree_local': 34.0, 'tree_grade': 9.0, 'mean_time': 28.239, 'min_time': 27.96, 'max_time': 28.63, 'by_mean_lost': 2.818, 'by_mean_win': 1.75, 'bends_mean': 2.969, 'position': 2.938, 'days_lr': 8.0, 'whelping': 1341.0, 'place': 2}, {'full': 2.0, 'trap': 1, 'distance': 71.0, 'local': 0.0, 'grade': 19.0, 'dist_trap': 6.0, 'dist_grade': 19.0, 'first_full': 1.0, 'first_trap': 7.0, 'first_dist': 31.0, 'first_local': 46.0, 'first_grade': 9.0, 'tree_full': 1.0, 'tree_trap': 9.0, 'tree_dist': 40.0, 'tree_local': 54.0, 'tree_grade': 10.0, 'mean_time': 28.161, 'min_time': 27.93, 'max_time': 28.66, 'by_mean_lost': 2.078, 'by_mean_win': 2.0, 'bends_mean': 3.934, 'position': 3.0, 'days_lr': 7.0, 'whelping': 1707.0, 'place': 3}, {'full': 0.0, 'trap': 3, 'distance': 31.0, 'local': 0.0, 'grade': 1.0, 'dist_trap': 13.0, 'dist_grade': 1.0, 'first_full': 0.0, 'first_trap': 5.0, 'first_dist': 10.0, 'first_local': 10.0, 'first_grade': 0.0, 'tree_full': 0.0, 'tree_trap': 8.0, 'tree_dist': 21.0, 'tree_local': 21.0, 'tree_grade': 1.0, 'mean_time': 28.89, 'min_time': 28.89, 'max_time': 28.89, 'by_mean_lost': 2.25, 'by_mean_win': 0.0, 'bends_mean': 4.5, 'position': 6.0, 'days_lr': 7.0, 'whelping': 878.0, 'place': 4}, {'full': 3.0, 'trap': 6, 'distance': 14.0, 'local': 0.0, 'grade': 3.0, 'dist_trap': 12.0, 'dist_grade': 3.0, 'first_full': 2.0, 'first_trap': 5.0, 'first_dist': 6.0, 'first_local': 9.0, 'first_grade': 2.0, 'tree_full': 1.0, 'tree_trap': 8.0, 'tree_dist': 8.0, 'tree_local': 13.0, 'tree_grade': 1.0, 'mean_time': 28.1, 'min_time': 27.99, 'max_time': 28.21, 'by_mean_lost': 1.75, 'by_mean_win': 3.0, 'bends_mean': 3.083, 'position': 2.667, 'days_lr': 4.0, 'whelping': 922.0, 'place': 5}, {'full': 0.0, 'trap': 5, 'distance': 5.0, 'local': 0.0, 'grade': 0.0, 'dist_trap': 4.0, 'dist_grade': 0.0, 'first_full': 0.0, 'first_trap': 3.0, 'first_dist': 4.0, 'first_local': 4.0, 'first_grade': 0.0, 'tree_full': 0.0, 'tree_trap': 1.0, 'tree_dist': 1.0, 'tree_local': 1.0, 'tree_grade': 0.0, 'mean_time': 0.0, 'min_time': 0.0, 'max_time': 0.0, 'by_mean_lost': 0.0, 'by_mean_win': 0.0, 'bends_mean': 0.0, 'position': 0.0, 'days_lr': 8.0, 'whelping': 513.0, 'place': 6}]
for i, s in enumerate(stats):    
    s_copy = s.copy()
    trap_a, position_a = s.copy().pop("trap"), s.copy().pop("place")
    s_copy.pop("trap")
    s_copy.pop("place")    
    for j, t in enumerate(stats):
        t_copy = t.copy()
        trap_b, position_b = t.copy().pop("trap"), t.copy().pop("place")
        t_copy.pop("trap")
        t_copy.pop("place")    
        if position_a != position_b:            
            #print(position_a, position_b)
            dog_a = np.array(list(s_copy.values()))
            dog_b = np.array(list(t_copy.values()))
            features = dog_a / (dog_a + dog_b)
            features = list(np.round(np.nan_to_num(features).astype(float), 3))
            features.append(trap_a)
            features.append(trap_b)
            if position_a < position_b:
                result = "A"
            elif position_a > position_b:
                result = "B"
            features.append(result)


1 2 A
1 3 A
1 4 A
1 5 A
1 6 A
2 1 B
2 3 A
2 4 A
2 5 A
2 6 A
3 1 B
3 2 B
3 4 A
3 5 A
3 6 A
4 1 B
4 2 B
4 3 B
4 5 A
4 6 A
5 1 B
5 2 B
5 3 B
5 4 B
5 6 A
6 1 B
6 2 B
6 3 B
6 4 B
6 5 B


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
